In [ ]:
!pip install langchain
!pip install langchain_together
!pip install langchain-community

In [48]:
import warnings
warnings.filterwarnings("ignore")

In [156]:
model_name="mistralai/Mistral-7B-Instruct-v0.2"

In [2]:
from getpass import getpass

key = getpass('Please enter your together AI API Key here: ')

Please enter your together AI API Key here: ········


In [90]:
import os

os.environ['TOGETHER_API_KEY'] = "b5356c7833ae679c0450b02475dc4ac9043ea10b6666d419f9608de6cc485fbf"

In [147]:
model="mistralai/Mistral-7B-Instruct-v0.2"


In [110]:
import os
from langchain_together import ChatTogether
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser


In [157]:
model = ChatTogether(model_name=model)


In [201]:
prompt = PromptTemplate(input_variables=["country","year"],
    template="tell me more about gdp of {country} in {year}"
)

In [198]:
prompt

PromptTemplate(input_variables=['country', 'year'], input_types={}, partial_variables={}, template='tell me more about gdp of {country} in {year}')

In [203]:
from langchain.chains import LLMChain
chain = LLMChain(llm=model, prompt=prompt, verbose=True)

print(chain.run({"country":"us","year":"2024"}))



> Entering new LLMChain chain...
Prompt after formatting:
tell me more about gdp of us in 2024

> Finished chain.
 I'm unable to provide a precise prediction about the GDP of the United States in 2024 as I don't have the ability to forecast economic data. However, I can tell you that the GDP of a country is influenced by a variety of factors such as government spending, consumer spending, investment, and net exports.

To get a sense of the current economic outlook, you can look at forecasts from reputable sources like the International Monetary Fund (IMF), the Organisation for Economic Co-operation and Development (OECD), or the U.S. Bureau of Economic Analysis. These organizations regularly publish predictions about GDP growth for various countries, including the United States.

As of my last update, the IMF predicts a 3.5% GDP growth rate for the United States in 2023. For 2024, they expect a slight slowdown to 2.3%. However, keep in mind that these forecasts are subject to change 

# Sequential Chain
A sequential chain is a chain that combines various individual chains, 
where the output of one chain serves as the input for the next in a continuous sequence. 
It operates by running a series of chains consecutively.

In [114]:

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SimpleSequentialChain

In [115]:
# This is an LLMChain to write first chain.
first_prompt = ChatPromptTemplate.from_template(
    " {product}?"
)
chain_one = LLMChain(llm=model, prompt=first_prompt)



In [116]:
# This is an LLMChain to write second chain.


second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company:{company_name}"
)
chain_two = LLMChain(llm=model, prompt=second_prompt)


In [117]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )
overall_simple_chain.run("gaming laptop")



> Entering new SimpleSequentialChain chain...
Here are some suggestions for a company name that makes gaming laptops:

1. **Pulse Gaming**: suggests energy, excitement, and a strong heartbeat, which is perfect for a gaming brand.
2. **Apex Computers**: implies a high level of performance, speed, and power, which is ideal for gaming laptops.
3. **Frenzy Systems**: conveys a sense of intense energy and excitement, which is often associated with gaming.
4. **Velocity Gaming**: emphasizes speed, agility, and quick reflexes, which are essential for gaming.
5. **Stormforge**: suggests a powerful and intense gaming experience, with a nod to the idea of forging powerful machines.
6. **RazorEdge Gaming**: implies a sharp, cutting-edge approach to gaming laptops, with a focus on performance and precision.
7. **Core Gaming Co.**: emphasizes the core components of a gaming laptop, such as the processor, graphics card, and RAM.
8. **Luminari Gaming**: suggests a bright, illuminated approach to ga

'High-performance gaming laptops designed for speed, power, and intense entertainment experiences always.'

In [118]:

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain

In [120]:
Review = """Les ordinateurs portables GamersTech impressionne par ses
 performances exceptionnelles et son design élégant. De sa configuration 
 matérielle robuste à un clavier RVB personnalisable et un système de 
 refroidissement efficace, il établit un équilibre parfait entre prouesses 
 de jeu et portabilité."""

# prompt template 1: translate to English

first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n \n {Review}"
)

# chain 1: input= Review and output= English_Review

chain_one = LLMChain(llm=model, prompt=first_prompt,
                     output_key="English_Review"                     
                    )
                   
                    

In [121]:
# prompt template 2: Summarize the English review

second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n {English_Review} "
)

# chain 2: input= English_Review and output= summary

chain_two = LLMChain(llm=model, prompt=second_prompt,
                     output_key="summary"
                    )


In [122]:
# prompt template 3: know review language

third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:"
    "\n\n{Review}"
)

# chain 3: input= Review and output= language

chain_three = LLMChain(llm=model, prompt=third_prompt,
                       output_key="language"
                      )


In [123]:
# prompt template 4: follow up message

fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}"
    "\n\nLanguage: {language}"
)

# chain 4: input= summary, language and output= followup_message

chain_four = LLMChain(llm=model, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [124]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","language","followup_message"],
    verbose=True
)
overall_chain(Review)




> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'Les ordinateurs portables GamersTech impressionne par ses\n performances exceptionnelles et son design élégant. De sa configuration \n matérielle robuste à un clavier RVB personnalisable et un système de \n refroidissement efficace, il établit un équilibre parfait entre prouesses \n de jeu et portabilité.',
 'English_Review': 'Here is the translation of the review to English:\n\nThe GamersTech laptops impress with their exceptional performance and elegant design. From its robust hardware configuration to a customizable RGB keyboard and an effective cooling system, it strikes a perfect balance between gaming prowess and portability.',
 'summary': 'The GamersTech laptop impresses with its exceptional performance, elegant design, and perfect balance between gaming capabilities and portability, thanks to its robust hardware, customizable RGB keyboard, and effective cooling system.',
 'language': 'The language of the review is French.',
 'followup_message': "Voici une réponse de

# LCEL

In [141]:
model = ChatTogether(model_name=model)
output_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_template(
    "Create a lively and engaging product description with emojis based on these notes: \n{product_notes}"
)


In [128]:
prompt_value = prompt.invoke({"product_notes": "Multi color affordable mobile covers"})


In [129]:
prompt_value

ChatPromptValue(messages=[HumanMessage(content='Create a lively and engaging product description with emojis based on these notes: \nMulti color affordable mobile covers', additional_kwargs={}, response_metadata={})])

In [130]:
prompt_value.to_string()


'Human: Create a lively and engaging product description with emojis based on these notes: \nMulti color affordable mobile covers'

In [131]:
prompt_value.to_messages()

[HumanMessage(content='Create a lively and engaging product description with emojis based on these notes: \nMulti color affordable mobile covers', additional_kwargs={}, response_metadata={})]

In [132]:
model_output = model.invoke(prompt_value.to_messages())


In [133]:
model_output

AIMessage(content="📱💥 Add a pop of color to your mobile with our vibrant and affordable mobile covers 🎉! 🌈 Choose from a RAINBOW of colors and designs to match your unique style 🤩. Our multi-color mobile covers are not only stylish but also super affordable 💸, so you can stay on-trend without breaking the bank 🤑.\n\n👉 Protect your phone in style with our durable and long-lasting covers 🛡️. They're designed to fit snugly around your device, providing maximum protection against scratches and bumps 🤯. Plus, they're easy to install and remove, making it a breeze to switch up your look whenever you want 🔄.\n\n👀 So why settle for a boring, plain cover when you can make a statement with our multi-color mobile covers 🤔? 🎁 Treat yourself or gift one to a friend or family member who loves to stay stylish and connected 📱. Order now and get ready to turn heads with your vibrant new mobile cover 😎! 💥", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 217,

In [134]:
print(output_parser.invoke(model_output))


📱💥 Add a pop of color to your mobile with our vibrant and affordable mobile covers 🎉! 🌈 Choose from a RAINBOW of colors and designs to match your unique style 🤩. Our multi-color mobile covers are not only stylish but also super affordable 💸, so you can stay on-trend without breaking the bank 🤑.

👉 Protect your phone in style with our durable and long-lasting covers 🛡️. They're designed to fit snugly around your device, providing maximum protection against scratches and bumps 🤯. Plus, they're easy to install and remove, making it a breeze to switch up your look whenever you want 🔄.

👀 So why settle for a boring, plain cover when you can make a statement with our multi-color mobile covers 🤔? 🎁 Treat yourself or gift one to a friend or family member who loves to stay stylish and connected 📱. Order now and get ready to turn heads with your vibrant new mobile cover 😎! 💥


In [142]:
chain = prompt | model | output_parser
product_description = chain.invoke({"product_notes": "Multi color affordable mobile covers"})
print(product_description)


🎉💻 Get ready to add a pop of color to your mobile phone with our vibrant and affordable multi-color mobile covers 📱👀! 🌈

Choose from a wide range of exciting colors and designs that will make your phone stand out from the crowd 🌴. Our mobile covers are not just stylish, but also durable and long-lasting 💪, protecting your device from scratches and damage 🤦‍♂️.

What's more, our multi-color mobile covers are super affordable 💸, so you can stay on-trend without breaking the bank 🤑. Whether you're a fan of bright and bold colors 🌟 or pastel shades 🌸, we've got you covered 📈.

So why settle for a boring, plain mobile cover when you can add some personality to your phone with our fun and colorful designs 🤔? 🎁 Treat yourself to a new mobile cover today and give your phone a fresh new look 📱💫!

Key features:
🔹 Affordable prices
🔹 Durable and long-lasting
🔹 Wide range of colors and designs
🔹 Protects your phone from scratches and damage

Get your multi-color mobile cover now and add some excit

In [151]:
# Streaming Example
for chunk in chain.stream({"product_notes": "Multi color affordable mobile covers"}):
    print(chunk, end="", flush=True)



🎉📱 Add a Pop of Color to Your Mobile with Our Affordable Multi-Color Covers 📱🎉

Elevate your mobile style with our vibrant and fun multi-color phone covers 🌈! Not only do they protect your device from scratches and damage 🛡️, but they also make a bold fashion statement 💃. Our affordable mobile covers come in a variety of colorful designs, ensuring you'll find the perfect match for your unique personality 🤩.

With our multi-color covers, you can:

✨ Express yourself with a wide range of colors and patterns
✨ Protect your phone from everyday wear and tear
✨ Stay on-trend with the latest mobile accessories
✨ Enjoy affordable prices without compromising on quality

So why settle for a boring, plain cover when you can make a statement with our multi-color mobile covers? 🤔 Treat yourself to a fresh new look and shop our collection today! 🛍️💰

👉 Key Features:
- Durable and long-lasting material
- Easy to install and remove
- Compatible with a variety of mobile models
- Affordable prices witho

In [149]:
# Batch Processing Example
product_notes_list = [
    {"product_notes": "Eco-friendly reusable water bottles"},
    #{"product_notes": "Eco-friendly reusable water bottles"},
    #{"product_notes":"Biodegradable plant-based straws"},
    #{"product_notes":"Solar-powered LED garden lights"},
    {"product_notes":"Recyclable bamboo toothbrushes"},
    {"product_notes":"Organic cotton tote bags"}]
batch_descriptions = chain.batch(product_notes_list)


In [150]:
for i,j in enumerate(batch_descriptions,1):
    print(i,j)

1 🌎💚 Stay hydrated and save the planet with our amazing eco-friendly reusable water bottles 🌟! 💦 Made from high-quality, BPA-free materials, these bottles are designed to keep your drinks fresh and hot or cold for hours 🕰️. 

🌈 Not only are they great for the environment, but they're also super stylish and come in a range of colors to match your unique vibe 🎨. Whether you're hitting the gym 🏋️‍♀️, going for a hike 🏞️, or just need a drink on-the-go, our reusable water bottles are the perfect companion 🤝.

🚫 Ditch single-use plastic bottles and make the switch to a more sustainable you 🌱. Our bottles are easy to clean, durable, and designed to last, so you can stay hydrated and reduce your waste 📉. 

👉 So why wait? Join the eco-friendly movement and get your hands on one of our fantastic reusable water bottles today 🎉! 💖 Your planet (and your taste buds) will thank you 🙏.
2 🌿🦷 Introducing the Eco-Friendly Game-Changer: Recyclable Bamboo Toothbrushes 🌸💚

Say goodbye to plastic waste and 